In [1]:
pip install streamlit plotly pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import streamlit as st
import mysql.connector
import pandas as pd
import plotly.express as px

# --------------------------
# DB CONFIG & HELPERS
# --------------------------
DB_CONFIG = {
    "host": "localhost",
    "user": "python_user",
    "password": "harshada",           # local dev only; normally use env vars
    "database": "security_analytics"
}

@st.cache_resource
def get_connection():
    return mysql.connector.connect(**DB_CONFIG)

def query_df(sql, params=None):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(sql, params or ())
    rows = cursor.fetchall()
    cols = [d[0] for d in cursor.description]
    cursor.close()
    # do NOT close conn here because cache_resource keeps it
    return pd.DataFrame(rows, columns=cols)

# --------------------------
# PAGE LAYOUT
# --------------------------
st.set_page_config(
    page_title="Smart Security Analytics Dashboard",
    layout="wide"
)

st.title("🔐 Smart Security Analytics Dashboard")
st.caption("Login activity, anomalies, and vulnerability exposure from simulated security logs.")

st.markdown("---")

# --------------------------
# TOP METRICS
# --------------------------

# Logins last 7 days
logins_7d = query_df("""
    SELECT
        COUNT(*) AS total_logins,
        SUM(CASE WHEN is_success = 0 THEN 1 ELSE 0 END) AS failed_logins
    FROM login_events
    WHERE event_time >= NOW() - INTERVAL 7 DAY;
""")

# Open vulnerabilities
vulns_open = query_df("""
    SELECT
        COUNT(*) AS open_vulns,
        SUM(CASE WHEN severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical_vulns
    FROM vulnerabilities
    WHERE remediated_at IS NULL;
""")

col1, col2, col3, col4 = st.columns(4)
with col1:
    st.metric("Logins (7 days)", int(logins_7d["total_logins"][0]))
with col2:
    st.metric("Failed Logins (7 days)", int(logins_7d["failed_logins"][0]))
with col3:
    st.metric("Open Vulnerabilities", int(vulns_open["open_vulns"][0]))
with col4:
    st.metric("Critical Open Vulns", int(vulns_open["critical_vulns"][0]))

st.markdown("---")

# --------------------------
# FAILED LOGIN TREND
# --------------------------
st.subheader("📈 Failed Login Trend (Daily)")

failed_trend = query_df("""
    SELECT
        DATE(event_time) AS login_date,
        COUNT(*) AS total_logins,
        SUM(CASE WHEN is_success = 0 THEN 1 ELSE 0 END) AS failed_logins
    FROM login_events
    GROUP BY DATE(event_time)
    ORDER BY login_date;
""")

fig_trend = px.line(
    failed_trend,
    x="login_date",
    y="failed_logins",
    title="Daily Failed Logins"
)
fig_trend.update_layout(xaxis_title="Date", yaxis_title="Failed Logins")

st.plotly_chart(fig_trend, use_container_width=True)

# --------------------------
# TOP USERS WITH FAILURES
# --------------------------
st.subheader("👤 Top Users by Failed Logins")

top_failed_users = query_df("""
    SELECT
        u.username,
        d.dept_name,
        COUNT(*) AS failed_count
    FROM login_events le
    JOIN users u ON le.user_id = u.user_id
    JOIN departments d ON u.dept_id = d.dept_id
    WHERE le.is_success = 0
    GROUP BY u.username, d.dept_name
    ORDER BY failed_count DESC
    LIMIT 10;
""")

fig_users = px.bar(
    top_failed_users,
    x="username",
    y="failed_count",
    hover_data=["dept_name"],
    title="Top 10 Users with Failed Logins"
)
fig_users.update_layout(xaxis_title="User", yaxis_title="Failed Logins")

st.plotly_chart(fig_users, use_container_width=True)

st.dataframe(top_failed_users)

# --------------------------
# VULNERABILITIES BY DEPARTMENT
# --------------------------
st.subheader("🛡️ Open Vulnerabilities by Department")

vulns_by_dept = query_df("""
    SELECT
        d.dept_name,
        COUNT(*) AS open_vulns,
        SUM(CASE WHEN v.severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical_vulns
    FROM vulnerabilities v
    JOIN departments d ON v.dept_id = d.dept_id
    WHERE v.remediated_at IS NULL
    GROUP BY d.dept_name
    ORDER BY open_vulns DESC;
""")

fig_vuln = px.bar(
    vulns_by_dept,
    x="dept_name",
    y="open_vulns",
    title="Open Vulnerabilities by Department"
)
fig_vuln.update_layout(xaxis_title="Department", yaxis_title="Open Vulnerabilities")

st.plotly_chart(fig_vuln, use_container_width=True)
st.dataframe(vulns_by_dept)

# --------------------------
# RAW TABLE EXPLORER
# --------------------------
st.markdown("---")
st.subheader("🔍 Explore Raw Tables")

table_choice = st.selectbox(
    "Choose a table to preview",
    ["login_events", "users", "departments", "vulnerabilities"]
)

preview_df = query_df(f"SELECT * FROM {table_choice} LIMIT 100;")
st.write(f"Showing first 100 rows of `{table_choice}`:")
st.dataframe(preview_df)


2025-11-10 16:20:28.976 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-10 16:20:29.130 Session state does not function when running a script without `streamlit run`


DeltaGenerator()